# A demo how to work with `QAOAParameter` classes

Farhi et al proposed in their original paper about QAOA the following Ansatz for the QAOA circuit:

$$
U = e^{-i \beta_p H_0} e^{-i \gamma_p H_1} \cdots e^{-i \beta_0 H_0} e^{-i \gamma  H_1}.
$$
...

### ToDo for this Demo
 - [ ] Finish the explanation of the different parametrizations
 - [ ] Explain _well_ what the `gammas_pairs`, `gammas_singles` and `betas` are in the raw format.
 - [ ] Demo _all_ different parametrizations that we have

In [1]:
# import the standard modules from python
import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Iterable

# import the neccesary pyquil modules
from pyquil.paulis import PauliSum, PauliTerm

# import the QAOAParameters that we want to demo
from qaoa.parameters import AdiabaticTimestepsQAOAParameters, AlternatingOperatorsQAOAParameters, AbstractQAOAParameters, GeneralQAOAParameters, QAOAParameterIterator, FourierQAOAParameters

## Creating the problem Hamiltonian

There are several ways of creating the problem Hamiltonian of interest. Ultimately, it should be in the form of a pyquil `PauliSum` object. See the documentation (http://docs.rigetti.com/en/stable/apidocs/pauli.html#) on `PauliSum` and `PauliTerm` objects here (the former is created from a list of the latter). 

For example, let's create the simple Hamiltonian
 
\begin{equation}
H = 0.7Z_0Z_1 + 1.2Z_0Z_2 -0.5Z_0
\end{equation}

Using `PauliTerm` and `PauliSum` directly, this can be implemented as follows.

In [2]:
# create a hamiltonian on 3 qubits with 2 coupling terms and 1 bias term
Term1 = PauliTerm("Z", 0, 0.7)*PauliTerm("Z", 1, 1.0)
Term2 = PauliTerm("Z", 0, 1.2)*PauliTerm("Z", 2, 1.0)
Term3 = PauliTerm("Z", 0, -0.5)

hamiltonian = PauliSum([Term1,Term2,Term3])
print("hamiltonian =", hamiltonian)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0


Another particularly easy way to create the Hamiltonian is using the `from_compact_str` method:

In [3]:
hamiltonian = PauliSum.from_compact_str("0.7*Z0*Z1 + 1.2*Z0*Z2 + (-0.5)*Z0")
print("hamiltonian =", hamiltonian)

hamiltonian = (0.7+0j)*Z0*Z1 + (1.2+0j)*Z0*Z2 + (-0.5+0j)*Z0


Having specified the problem Hamiltonian, we next move on to defining the QAOA parameters we want to use in our quantum circuit.

## QAOA parameter classes

While with the VQE we must find both good circuit ansatze and good parametrisations, in QAOA the former is fixed to be the alternate application of the mixer and cost operators (sometimes also referred to as the reference and phase separation operators, respectively). The goal is then to find the parameters that optimise the cost function when evaluated with resepct to the quantum state produced by the circuit. 

We have a considerable degree of flexibility in choosing how to parametrise a QAOA problem. We can choose a smaller set of parameters, where the optimisation landscape has lower dimension at the expense of reduced expressivity; or we can choose a larger set, where we can generate a wider set of quantum states with lower circuit depth, but the corresponding optimisation landscape has a higher dimension. The larger the set of parameters we have to optimise over, the more difficult it can become to find the optimal solution.

In general, the QAOA consists of two sets of parameters: hyperparameters and variable parameters. The hyperparameters may in turn be divided into two sets.

1) Those originating from the cost Hamiltonian: the qubit register (the qubits to be used in the algorithm); the qubits with a bias term (i.e. their own Z term in the Hamiltonian), and the corresponding coefficients; and the qubit pairs that have an interaction, along with the corresponding coefficients.

2) The number of QAOA steps we wish to perform - i.e. the QAOA p parameter.

The variable parameters are those we wish to optimise, which in turn depend on the specific parametrisation for the circuit we have chosen. We provide more details of the available parametrisations in a separate document, however note that:

- The `StandardParams` class implements the original form of the QAOA, as described in [Ref Farhi et al]. In time step $q$ of the algorithm, the mixer and cost Hamiltonians are applied with coefficients $\beta^{(q)}$ and $\gamma^{(q)}$, respectively, giving a total of $2p$ parameters over which we need to optimise. 

    GIVE AN EXAMPLE UNITARY OPERATOR

- For the `ExtendedParameters` class, each operator in both the cost and mixer Hamiltonians has its own angle, so that the set of variable parameters are:

    - $\mbox{betas} = \{\beta_0^{(1)},...,\beta_{n-1}^{(1)},\beta_0^{(2)},...,\beta_{n-1}^{(2)},...,\beta_0^{(p)},...,\beta_{n-1}^{(p)}\}$, where $\beta_i^{(q)}$ denotes the mixer Hamiltonian angle for qubit $i$ in the QAOA step $q$.
    - $\mbox{gammas_singles} = \left\{ \{\gamma_s^{(1)}\}, \{\gamma_s^{(2)}\},...,\{\gamma_s^{(p)}\}    \right\}$, where  where $s$ is the set of qubits with bias terms in the cost Hamiltonian, and $\{\gamma_s^{(q)}\}$ denotes the set of angles corresponding to those bias terms in QAOA step $q$.  
    - $\mbox{gammas_pairs} = \left\{ \{\gamma_{\Pi}^{(1)}\}, \{\gamma_{\Pi}^{(2)}\},...,\{\gamma_{\Pi}^{(p)}\}    \right\}$, where  where $\Pi$ is the set of qubits with bias terms in the cost Hamiltonian, and $\{\gamma_{\Pi}^{(q)}\}$ denotes the set of angles corresponding to those bias terms in QAOA step $q$.  
    
    GIVE AN EXAMPLE UNITARY OPERATOR
    
We currently provide two additional parameter classes that may be of interest, either for didactic or practical purposes.
    
- `AdiabaticParams` [Explanation and demo to be completed]

- `FourierParams` [Explanation and demo to be completed]

### Building parameters from the `AbstractQAOAParameter` class

All of the QAOA parameter classes listed above are descendants of the parent class `AbstractQAOAParameters`. An object belonging to this class is characterised by the problem hyperparameters. We create such an object as follows:

In [4]:
timesteps = 2
myAbstractParams = AbstractQAOAParameters([hamiltonian,timesteps])
print(myAbstractParams)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	single_qubit_coeffs: [-0.5]
	qubits_pairs: [[0, 1], [0, 2]]
	pair_qubit_coeffs: [0.7 1.2]
	timesteps: 2



Subsequently, we can initalise any of the sets of variable parameters described above by making use of the `AbstractQAOAParameters` object. For example, let's set up `GeneralQAOAParameters` using the `AbstractQAOAParameters` we have just defined. To do this, we call the `.from_AbstractParameters` method.

In [5]:
# Specify some angles
betas          = [[0.0, 0.1, 0.3], [0.5, 0.2, 1.2]]
gammas_singles = [[0.0], [0.5]]
gammas_pairs   = [[0.1, 0.3], [0.2, 1.2]]
parameters = (betas, gammas_singles, gammas_pairs)

myGeneralParams = GeneralQAOAParameters.from_AbstractParameters(myAbstractParams,parameters)
print(myGeneralParams)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [[0.  0.1 0.3], [0.5 0.2 1.2]]
	gammas_singles: [[0. ], [0.5]]
	gammas_pairs: [[0.1 0.3], [0.2 1.2]]



Note that we can also create `GeneralQAOAParameters` directly, without first setting up `AbstractQAOAParameters`. For the case considered above, we would simply do the following:

In [6]:
myGeneralParams_direct = GeneralQAOAParameters([hamiltonian,2],parameters)
print(myGeneralParams_direct)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [[0.  0.1 0.3], [0.5 0.2 1.2]]
	gammas_singles: [[0. ], [0.5]]
	gammas_pairs: [[0.1 0.3], [0.2 1.2]]



We could set up an object of any of the other parameter classes analogously to the case we have illustrated above for `GeneralQAOAParameters`, either directly, or through `AbstractQAOAParameters`. The benefit of the latter method is that it allows multiple different parametrisations to be set up from the same underlying set of `hyperparameters`.


For all parameter classes, we also provide the `.linear_ramp_from_hamiltonian()` method, which automatically produces a set of parameters obeying a linear ramp for the number of timesteps specified. [TODO: EXPLAIN THIS IN MORE DETAIL HERE OR ELSEWHERE]. For example:

In [7]:
timesteps = 2      # equivalent to p in Farhi's original QAOA paper
time = 1            # total time of the annealing schedule
params = AlternatingOperatorsQAOAParameters\
            .linear_ramp_from_hamiltonian(hamiltonian, timesteps, time = time) 
print(params)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.375 0.125]
	gammas_singles: [0.125 0.375]
	gammas_pairs: [0.125 0.375]



[@JANLUKAS: Should we include an explanation somewhere of the factor of 0.7 used to define the angles? Otherwise the above may look a little weird to the user at first]


We see that if we print `params`, it gives us the timesteps, the angles, and on which combinations of qubits they act. But we can also have a look at the _internal_ `x_rotation_angles`, `z_rotation_angles` and `zz_rotation_angles` that were automatically calculated under the hood. These are the rotation angles for the different Pauli operators in the actual execution of the quantum circuit, i.e. in the time evolution operator (the exponentiated Hamiltonian); they are determined by products of the `parameters` and the `coefficients` in the Hamiltonian itself [TODO: REPLACE `coefficients` WITH THE ACTUAL NAMES OF THE PARAMS. ALSO WORD THIS BETTER TO MAKE CLEAR WHAT HAPPENS WITH THE MIXER HAMILTONIAN].

In [8]:
print("\n x_rotation_angles:\n", params.x_rotation_angles)
print("\n z_rotation_angles:\n", params.z_rotation_angles)
print("\n zz_rotation_angles:\n", params.zz_rotation_angles)


 x_rotation_angles:
 [[0.375 0.375 0.375]
 [0.125 0.125 0.125]]

 z_rotation_angles:
 [[-0.0625]
 [-0.1875]]

 zz_rotation_angles:
 [[0.0875 0.15  ]
 [0.2625 0.45  ]]


In a standard workflow, we would then pass these `params` to a cost_function that has been built with `qaoa.cost_function.QAOACostFunctionOnQVM` or with `qaoa.cost_function.QAOACostFunctionOnWFSim`. To do so, we first arrange the `params` in a single 'raw' list of parameters in the form [`betas`,`gammas_singles`,`gammas_pairs`]:

In [9]:
raw_params = params.raw()
raw_params

array([0.375, 0.125, 0.125, 0.375, 0.125, 0.375])

## What happens when an invalid set of parameters are passed to a `QAOAParameter` class?

Here give an example of what happens if the user specifies too many or too few parameters for the choice of parametrisation they want to use...

## Modifying parameters

We may wish to modify the parameters that have been set up, before running QAOA with them. The two ways we can do this are to modify the `params` or the `raw_params`.

In [10]:
# Current params and under-the-hood equivalents
print(params)
print("\n x_rotation_angles:\n", params.x_rotation_angles)
print("\n z_rotation_angles:\n", params.z_rotation_angles)
print("\n zz_rotation_angles:\n", params.zz_rotation_angles)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [0.375 0.125]
	gammas_singles: [0.125 0.375]
	gammas_pairs: [0.125 0.375]


 x_rotation_angles:
 [[0.375 0.375 0.375]
 [0.125 0.125 0.125]]

 z_rotation_angles:
 [[-0.0625]
 [-0.1875]]

 zz_rotation_angles:
 [[0.0875 0.15  ]
 [0.2625 0.45  ]]


Let's modify the first `betas` parameter:

In [11]:
params.betas[0] = np.pi
print(params)
print("\n x_rotation_angles:\n", params.x_rotation_angles)
print("\n z_rotation_angles:\n", params.z_rotation_angles)
print("\n zz_rotation_angles:\n", params.zz_rotation_angles)

Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.14159265 0.125     ]
	gammas_singles: [0.125 0.375]
	gammas_pairs: [0.125 0.375]


 x_rotation_angles:
 [[3.14159265 3.14159265 3.14159265]
 [0.125      0.125      0.125     ]]

 z_rotation_angles:
 [[-0.0625]
 [-0.1875]]

 zz_rotation_angles:
 [[0.0875 0.15  ]
 [0.2625 0.45  ]]


If we know the index of the parameter(s) we wish to vary in the `raw` list, we can also modify them there.

## Iterating over parameter ranges

We may sometimes want to investigate what happens if we vary one of the parameters over some specfied range, keeping all others fixed. The `QAOAParameterIterator` class gives a convenient way of doing this.

For the parameters above, we could for example take one of the gammas_singles to iterate over. Suppose we take the second one, gammas_singles[1], and we want to vary it in steps of size 1/3 between 0 and 1:

In [12]:
the_range = np.arange(0, 1, 0.3334)
the_parameter = "gammas_singles[1]"
iterator= QAOAParameterIterator(params, the_parameter, the_range)
for i,p in zip(range(len(the_range)),iterator):
    print('Parameters at step' + str(i) + ':')
    print(p)

Parameters at step0:
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.14159265 0.125     ]
	gammas_singles: [0.125 0.   ]
	gammas_pairs: [0.125 0.375]

Parameters at step1:
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.14159265 0.125     ]
	gammas_singles: [0.125  0.3334]
	gammas_pairs: [0.125 0.375]

Parameters at step2:
Hyperparameters:
	register: [0, 1, 2]
	qubits_singles: [0]
	qubits_pairs: [[0, 1], [0, 2]]
Parameters:
	betas: [3.14159265 0.125     ]
	gammas_singles: [0.125  0.6668]
	gammas_pairs: [0.125 0.375]



As expected, in the loop all parameters stay constant except for `gammas_singles[1]` which went up in steps of thirds. 

Of course, we can also nest these iterators to sweep two-dimensional paramter landscapes. For instance, suppose we now sweep over parameter ranges for both `gammas_singles[1]` and `gammas_pairs[0]`:

In [13]:
range1 = np.arange(0,1,0.5)
range2 = np.arange(100,101,0.5)
for j in QAOAParameterIterator(params, "gammas_singles[1]", range1):
    for p in QAOAParameterIterator(j, "gammas_pairs[0]", range2):
        print(p.gammas_singles)
        print(p.gammas_pairs)

[0.125 0.   ]
[100.      0.375]
[0.125 0.   ]
[100.5     0.375]
[0.125 0.5  ]
[100.      0.375]
[0.125 0.5  ]
[100.5     0.375]


We can nest iterators arbitrarily, which allows for parameter sweeps and subsequent landscape analysis of any desired set of parameters. This functionality will be demoed in a separate notebook.

In [14]:
# Sweep over three parameters

iterator1 = QAOAParameterIterator(params,"betas[0]",np.arange(0,1,0.5))
for p1 in iterator1:
    iterator2 = QAOAParameterIterator(p1,"betas[1]",np.arange(0,1,0.5))
    for p2 in iterator2:
        iterator3 = QAOAParameterIterator(p2,"gammas_pairs[0]",np.arange(0,1,0.5))
        for j in iterator3:
            print(j.betas,j.gammas_pairs)
            

[0. 0.] [0.    0.375]
[0. 0.] [0.5   0.375]
[0.  0.5] [0.    0.375]
[0.  0.5] [0.5   0.375]
[0.5 0. ] [0.    0.375]
[0.5 0. ] [0.5   0.375]
[0.5 0.5] [0.    0.375]
[0.5 0.5] [0.5   0.375]
